# Exercise: change a tuning parameter


<div class="alert alert-info">
<strong>Exercise: Modify a parameter, dcs</strong><br><br>
 

In the tuning lecture, we talked about the parameter dcs:
http://www.cesm.ucar.edu/events/tutorials/2019/files/Specialized-hannay.pdf

Create a case called “b1850_dcs” using the compset  B1850  at f19_g17 resolution. 

Locate where to change the parameter Dcs and change from the default value:
dcs = 200.D-6 
to 
dcs = 500.D-6 

Make a 1-month run.

</div>



<div class="alert alert-warning">  
<details>
   <summary>
      <font face="Times New Roman" color='blue'>Click here for hints</font>
    </summary>

- The trick is going to locate where to change dcs 
    
- Compare to this run to the first run you did today: b1850_high_freq.
-  You can use ncdiff and ncview to look at the difference between the 2 runs.
```
ncdiff /glade/scratch/$user/archive/b1850_dcs/atm/hist/b1850_dcs.cam.h0.0001-01.nc /glade/scratch/$user/archive/b1850_high_freq/atm/hist/b1850_high_freq.cam.h0.0001-01.nc diff.nc

ncview diff.nc
```
- Analyze how this does this affect the LWCF ?

</details>
</div>



<div class="alert alert-success">   
<details>
<summary><font face="Times New Roman" color='blue'>Click here for the solution</font></summary><br>
                
Create a new case <font face="Courier" color='purple'><strong>b1850_dcs</strong></font> with the command:
```
cd /glade/p/cesm/tutorial/cesm2.1_tutorial2022/cime/scripts
./create_newcase --case ~/cases/b1850_dcs --compset B1850 --res f19_g17 
```

Case <font face="Courier" color='purple'><strong>setup</strong></font>:
```    
cd ~/cases/b1850_dcs
./case.setup
```
    
Locate the file where <font face="Courier" color='purple'><strong>T500</strong></font> 
is computed and copy it into <font face="Courier" color='purple'><strong>SourceMods/src.atm</strong></font>:
```
cp /glade/p/cesm/tutorial/cesm2.1_tutorial2022/components/cam/src/physics/cam/cam_diagnostics.F90 SourceMods/src.cam
```
    
Edit the file <font face="Courier" color='purple'><strong>SourceMods/src.cam/cam_diagnostics.F90</strong></font> and add the lines:
```   
!++ add a variable for T750 
call addfld ('T750',       horiz_only,  'A', 'K','Temperature at 750 mbar pressure surface')  
    
!++ add a variable for T750 
if (hist_fld_active('T750')) then 
    call vertinterp(ncol, pcols, pver, state%pmid, 75000._r8, state%t, p_surf, & 
    extrapolate='T', ps=state%ps, phis=state%phis)
    call outfld('T750    ', p_surf, pcols, lchnk )
end if
```
    
Edit the file <font face="Courier" color='purple'><strong>user_nl_cam</strong></font>  
    and add the lines:
```
nhtfrq = 0, -24
mfilt = 1, 31
fincl2 = 'T750', 'T500'
```
    
Change the <font face="Courier" color='purple'><strong>run length </strong></font>:
```   
./xmlchange STOP_N=1,STOP_OPTION=nmonths
```

If needed, change <font face="Courier" color='purple'><strong>job queue</strong></font> 
and <font face="Courier" color='purple'><strong>account number</strong></font>. 
For instance:
```  
./xmlchange JOB_QUEUE=regular,PROJECT=UESM0011
```

Build and submit:
```
qcmd -- ./case.build
./case.submit
```
    
When the run is completed, check the fields <font face="Courier"color='purple'><strong>T750</strong></font> 
    and <font face="Courier"color='purple'><strong>T500</strong></font> are in the file 
    <font face="Courier"color='purple'><strong>h1</strong></font>:
```  
cd /glade/scratch/$user/archive/b1850_dcs/atm/hist/ 
ncdump -h b1850_dcs.cam.h1.0001-01-01-00000.nc
```
    
The file should contain:
```
float T500(time, lat, lon) ;
    T500:units = "K" ;
    T500:long_name = "Temperature at 500 mbar pressure surface" ;
    T500:cell_methods = "time: mean" ;
float T750(time, lat, lon) ;
    T750:units = "K" ;
    T750:long_name = "Temperature at 750 mbar pressure surface" ;
    T750:cell_methods = "time: mean" ;
```
If you don't see these variables, check you correctly set the user_nl_cam.
    
Create a file with the difference between <font face="Courier" color='purple'><strong>T750-T500 </strong>:</font>
```    
cd /glade/scratch/$user/archive/b1850_dcs/atm/hist/ 
ncap2 -s ‘T750_minus_T500=T750-T500' b1850_dcs.cam.h1.0001-01-01-00000.nc  T750-T500.nc 
```

Look at the difference between <font face="Courier" color='purple'><strong>T750-T500</strong></font> 
    with <font face="Courier" color='purple'><strong>ncview</strong></font>: 
```
    cd /glade/scratch/$user/archive/b1850_dcs/atm/hist/ 
    ncview T750-T500.nc
```
  
The field <font face="Courier" color='purple'><strong>T750-T500</strong></font> looks like:

*<p style="text-align: center;"> 
    ![ncview T750-T500](../../images/sourcemods/ncview_T750-T500.png)  
    Figure: Overview of the CESM directories and the SourceMods directories. </p>*
    
</details>
</div>

